# Summary
Implemented CCNY Calendar scraper by using the requests library to retrieve the HTML tags, then used BS4 to extract each table row from the tbody found, which corresponds to schedule. Simply stored the text of each table data into the DataFrame with the expected column names.

In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [68]:
# get page:
ccny_page = 'https://www.ccny.cuny.edu/registrar/fall'
r = requests.get(ccny_page)
soup = BeautifulSoup(r.text, 'html.parser')
soup.prettify()
table = soup.find('tbody') # table contains all the table contents

In [69]:
# FUNCTION for converting scraped date -> Python datetime
def conv_date(strng):
    # two cases: single day or range days
    if '2022' in strng:
        date_str = strng.replace(',', '')
    else:
        date_str = strng + " 2021"
    if '-' not in strng: # make sure to also consider that final day "January 1, 2022" which has diff format
        return [datetime.strptime(date_str, "%B %d %Y").date()]
    full_ranges = strng.split(' - ') # should just be two values always since its a range
    start = datetime.strptime(full_ranges[0] + " 2021", "%B %d %Y").date()
    # reuse start for end
    end = start.replace(day=int(full_ranges[1]))
    return [str(start) + " to " + str(end)]

In [70]:
# BUILDING the DataFrame table off scraping:
temp_data = []
for tr in table.find_all('tr'):
    dates = tr.find('td', {'style': 'width:305px'})
    dow = tr.find('td', {'style': 'width:327px'})
    events = tr.find('td', {'style': 'width:617px'})
    dates = dates.get_text(strip=True)
    dow = dow.get_text(strip=True)
    events = events.get_text(strip=True)
    temp_data.append([dates, dow, events])
df_ccny_cal = pd.DataFrame(temp_data, columns=['dates', 'dow', 'text'])
df_ccny_cal["dates"] = df_ccny_cal["dates"].apply(conv_date)
df_ccny_cal.set_index("dates", inplace=True)
df_ccny_cal.head(50)

,dow,text
dates,,
[2021-08-01],Sunday,Application for degree for January and Februar...
[2021-08-18],Wednesday,Last day to apply for Study Abroad
[2021-08-24],Tuesday,Last day of Registration;Last day to file ePer...
[2021-08-25],Wednesday,Start of Fall Term;Classes begin;Initial Regis...
[2021-08-25 to 2021-08-31],Wednesday - Tuesday,Change of program period; late fees apply
[2021-08-26],Thursday,Last day for Independent Study
[2021-08-28],Saturday,First day of Saturday Classes
[2021-08-31],Tuesday,Last day to add a class to an existing enrollm...
[2021-09-01],Wednesday,Verification of Enrollment rosters available t...
